In [1]:
2800*6.81

19068.0

In [ ]:
import astropy
import numpy as np
import pandas as pd
from astropy.cosmology import FlatLambdaCDM

#PARAMETERS

#COSMOLOGY
om = 0.3089
h = 0.6774

#BOX SIZE
lbox = 3000
off = 0.1

#DIRECTION VECTOR OF THE LIGHTCONE
ra = np.radians(31) # RA FOR DIRECTION VECTOR
piby2_minus_dec  = np.radians(80)  ## 80 is the theta in spherical coordinates DEC FOR DIRECTION VECTOR

#BUFFER REGION
buffer_comovD = 1

#AREA COVERAGE (degrees^2)
A = 50

#REDSHIFTS OF THE USED SNAPSHOTS
z=[16.98,15.67,15.31,14.95,14.6,14.24,13.93,13.6,13.27,12.97,12.66,12.35,12.05,11.77,11.5,11.22,10.96,10.7,10.44,10.19,9.941,9.707,9.471,9.235,9.01,8.794,8.579,8.372,8.166,7.961,7.764,7.576,7.389,7.203,7.026,6.849,6.675,6.508,6.342,6.184,6.022,5.873,5.72,5.575,5.431,5.289,5.15,5.017,4.882,4.754,4.627,4.507,4.385,4.266,4.152,4.038,3.929,3.819,3.715,3.61,3.511,3.411,3.314,3.221,3.127,3.037,2.949,2.862,2.778,2.695,2.614,2.535,2.458,2.382,2.308,2.235,2.165,2.095,2.028,1.961,1.896,1.833,1.771,1.71,1.65,1.593,1.535,1.48,1.425,1.372,1.321,1.27,1.22,1.172,1.124,1.077,1.032,0.9873,0.9436,0.9011,0.8594,0.8188,0.7787,0.74,0.7018,0.6644,0.6281,0.5924,0.5574,0.5232,0.4899,0.4573,0.4253,0.3941,0.3637,0.3337,0.3045,0.276,0.248,0.2207,0.1939,0.1677,0.1422,0.1172,0.09266,0.06872,0.04526,0.02239,0.00000]
redshift_table = './redshift_list.txt'

#OUTPUT TABLE NAME
out_name = 'join_table_test'

###################################
###################################

###UTILS:
def generate_grid_points_in_volume(xmin, xmax, ymin, ymax, zmin, zmax):
    x = np.linspace(xmin, xmax,400) #WHY 400??
    y = np.linspace(ymin, ymax,400)
    z = np.linspace(zmin, zmax,400)
    xx,yy,zz = np.meshgrid(x,y,z)
    flattened_grid = np.column_stack([xx.flatten(), yy.flatten(),zz.flatten()])
    return flattened_grid.T

def generate_redshift(redshift_list):
    with open(redshift_list, "r") as file:
        redshift_values = [float(line.strip()) for line in file if line.strip()]

    redshift_values.sort()

    redshift_dict = {f"Z={z}": [z] for z in redshift_values}
    return redshift_dict

###################################
cosmo = FlatLambdaCDM(H0=100*h, Om0=om)
nsnaps = len(np.genfromtxt(redshift_table))
#nsnaps = len(redshift_table)

###CREATE REDSHIFT TABLE FORM: redshift = {"Z=0":[0],  etc....}

redshift = generate_redshift(redshift_table)
print(redshift)

####COMOVING DISTANCE:
for i, label in enumerate(redshift.keys()):
    redshift[label].append(h*cosmo.comoving_distance(redshift[label][0]).value)

comov_dist = np.zeros([nsnaps])
redshift_array = np.zeros([nsnaps])

for i,snapshot in enumerate(redshift):
    comov_dist[i] = h*cosmo.comoving_distance(redshift[snapshot][0]).value
    redshift_array[i] = redshift[snapshot][0]

comov_dist_mid = np.pad((comov_dist[1:]+comov_dist[0:-1])/2,(1,1))
comov_dist_mid[-1] = comov_dist[-1]

####GENERATE GRID POINTS (HERE WE ARE ASSUMING WE ARE USING THE SAME GEOMETRY; CHANGE IF NEEDED)!
points1 = generate_grid_points_in_volume(0, lbox, 0, lbox, 0, lbox)+off #IMPROVE THIS
points2 = generate_grid_points_in_volume(lbox, lbox+lbox, 0, lbox, 0, lbox)+off
points3 = generate_grid_points_in_volume(lbox, lbox+lbox, lbox, lbox+lbox, 0, lbox)+off
points4 = generate_grid_points_in_volume(lbox+lbox, lbox + 2*lbox, lbox, lbox+lbox, 0, lbox)+off

####COMPUTE DISTANCES
points_r1 = np.sqrt(points1[0]**2+points1[1]**2+points1[2]**2)
points_r2 = np.sqrt(points2[0]**2+points2[1]**2+points2[2]**2)
points_r3 = np.sqrt(points3[0]**2+points3[1]**2+points3[2]**2)
points_r4 = np.sqrt(points4[0]**2+points4[1]**2+points4[2]**2)

####CREATE DIRECTION VECTOR
direction_vector=[np.cos(ra)*np.sin(piby2_minus_dec),np.sin(ra)*np.sin(piby2_minus_dec),np.cos(piby2_minus_dec)]

####ANGLES TO DIRECTION VECTOR
cosangle1 = np.dot(points1.T,direction_vector)/points_r1
cosangle2 = np.dot(points2.T,direction_vector)/points_r2
cosangle3 = np.dot(points3.T,direction_vector)/points_r3
cosangle4 = np.dot(points4.T,direction_vector)/points_r4

######CREATE TABLE ITSELF:
columns = ['comovD_Min(Mpchinv)', 'comovD_Max(Mpchinv)','theta(radians)','buffer_comovD(Mpchinv)','buffer_theta(radians)','direction_vector_x','direction_vector_y','direction_vector_z','Kind_of_join', 'halodir1','halodir2','field1','field2','Box1(w/ buffer)','Box2(w/ buffer)','Box3(w/ buffer)','Box4(w/ buffer)']

df = pd.DataFrame(0,columns=columns,index=np.arange((nsnaps-1)*2))

cos_theta_max =  1-(A/2)*np.pi/180**2 ## theta is the angle made with the cone
theta = np.arccos(cos_theta_max) ## in radians

df['Kind_of_join'] = 'left'
df['theta(radians)'] = theta
df['buffer_comovD(Mpchinv)'] = buffer_comovD
df['direction_vector_x'] = direction_vector[0]
df['direction_vector_y'] = direction_vector[1]
df['direction_vector_z'] = direction_vector[2]
for i in range(nsnaps-1):
    df.at[2*i,'comovD_Min(Mpchinv)'] = comov_dist[i]
    df.at[2*i,'comovD_Max(Mpchinv)'] = comov_dist_mid[i+1]
    df.at[2*i,'halodir1'] = nsnaps-i
    df.at[2*i,'halodir2'] = nsnaps-i-1
    df.at[2*i,'field1'] = 'last_mainleaf_depthfirst_id'
    df.at[2*i,'field2'] = 'last_mainleaf_depthfirst_id'
    buffer_theta = 1/(comov_dist_mid[i]+5)
    df.at[2*i,'buffer_theta(radians)'] = buffer_theta
    df.at[2*i,'Box1(w/ buffer)'] = ( ((cosangle1>np.cos(theta+buffer_theta))&(points_r1>comov_dist[i]-buffer_comovD)&(points_r1<comov_dist_mid[i+1]+buffer_comovD)).any())
    df.at[2*i,'Box2(w/ buffer)'] = ( ((cosangle2>np.cos(theta+buffer_theta))&(points_r2>comov_dist[i]-buffer_comovD)&(points_r2<comov_dist_mid[i+1]+buffer_comovD)).any())
    df.at[2*i,'Box3(w/ buffer)'] = ( ((cosangle3>np.cos(theta+buffer_theta))&(points_r3>comov_dist[i]-buffer_comovD)&(points_r3<comov_dist_mid[i+1]+buffer_comovD)).any())
    df.at[2*i,'Box4(w/ buffer)'] = ( ((cosangle4>np.cos(theta+buffer_theta))&(points_r4>comov_dist[i]-buffer_comovD)&(points_r4<comov_dist_mid[i+1]+buffer_comovD)).any())
    
    df.at[2*i + 1, 'comovD_Min(Mpchinv)'] = comov_dist_mid[i+1]
    df.at[2*i + 1, 'comovD_Max(Mpchinv)'] = comov_dist[i+1]
    df.at[2*i + 1, 'halodir1'] = nsnaps-i-1
    df.at[2*i + 1, 'halodir2'] =nsnaps-i
    df.at[2*i + 1, 'field1'] = 'desc_id'
    df.at[2*i + 1, 'field2'] = 'id'
    buffer_theta = 1/(comov_dist_mid[i+1]+5)
    df.at[2*i + 1,'buffer_theta(radians)'] = buffer_theta
    df.at[2*i + 1, 'Box1(w/ buffer)'] = ( ((cosangle1>np.cos(theta+buffer_theta))&(points_r1>comov_dist_mid[i+1]-buffer_comovD)&(points_r1<comov_dist[i+1]+buffer_comovD)).any())
    df.at[2*i + 1, 'Box2(w/ buffer)'] = ( ((cosangle2>np.cos(theta+buffer_theta))&(points_r2>comov_dist_mid[i+1]-buffer_comovD)&(points_r2<comov_dist[i+1]+buffer_comovD)).any())
    df.at[2*i + 1, 'Box3(w/ buffer)'] = ( ((cosangle3>np.cos(theta+buffer_theta))&(points_r3>comov_dist_mid[i+1]-buffer_comovD)&(points_r3<comov_dist[i+1]+buffer_comovD)).any())
    df.at[2*i + 1, 'Box4(w/ buffer)'] = ( ((cosangle4>np.cos(theta+buffer_theta))&(points_r4>comov_dist_mid[i+1]-buffer_comovD)&(points_r4<comov_dist[i+1]+buffer_comovD)).any())

df.to_pickle('./' +out_name+ '.pkl')
